In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
from PIL import Image

In [15]:
test_img = Image.open("./goldfish.jpg")
test_img = test_img.resize((224, 224))
transform = torchvision.transforms.ToTensor()
test_img = transform(test_img).unsqueeze(0)

In [2]:
model = torchvision.models.vgg16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

layers = model.features[:4]

In [ ]:
def reg(noise):
    noise = noise.squeeze(0).permute(1,2,0)
    reg_loss = 0
    for i in range(224 - 2):
        for j in range(224 - 2):
            reg_loss += (noise[i, j+1] - noise[i, j])**2 + (noise[i+1, j] - noise[i, j])**2
    return reg_loss


In [ ]:
# put a image into model and get output from
given_feature = layers(x)

# init a random noise, gradient descent on it to reconstruct the image.
noise = torch.randn(1, 3, 224, 224)
optimizer = torch.optim.Adam(noise)
criterion = nn.MSELoss()

for i in range(100):
    noise_feature = layers(noise)

    # loss has two parts: 1. the L2 distance between feature 2. regularizer term
    loss = criterion(given_feature, noise_feature) + reg(noise)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        plt.imshow(noise.squeeze(0).permute(1,2,0).numpy())
        plt.title(f"epoch({i})")
        plt.axis("off")
        plt.imshow()
